# Support Vector Machines

This notebook contains sample code to classify digits using the MNIST Dataset and the solutions to **Chapter 5 assignment.**

A Support Vector Machine (SVM) is a powerful and versatile Machine Learning
model, capable of performing linear or nonlinear classification, regression, and even
outlier detection. It is one of the most popular models in Machine Learning, and any‐
one interested in Machine Learning should have it in their toolbox. SVMs are partic‐
ularly well suited for classification of complex small- or medium-sized datasets.

This chapter will explain the core concepts of SVMs, how to use them, and how they
work.

**About Dataset:**

The MNIST dataset contains 70,000 small images of digits handwritten by high school students and employees of the US Census Bureau. Each image is labeled with the digit it represents.

**Credit:**

O'Reilly book Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow

# Assignment 5

1. What is the fundamental idea behind Support Vector Machines?
2. What is a support vector?
3. Why is it important to scale the inputs when using SVMs?
4. Can an SVM classifier output a confidence score when it classifies an instance?
What about a probability?
5. Should you use the primal or the dual form of the SVM problem to train a model
on a training set with millions of instances and hundreds of features?
6. Say you’ve trained an SVM classifier with an RBF kernel, but it seems to underfit
the training set. Should you increase or decrease γ (gamma)? What about C?
7. How should you set the QP parameters (H, f, A, and b) to solve the soft margin
linear SVM classifier problem using an off-the-shelf QP solver?
8. Train a LinearSVC on a linearly separable dataset. Then train an SVC and a
SGDClassifier on the same dataset. See if you can get them to produce roughly
the same model.
9. Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary
classifiers, you will need to use one-versus-the-rest to classify all 10 digits. You
may want to tune the hyperparameters using small validation sets to speed up the
process. What accuracy can you reach?
10. Train an SVM regressor on the California housing dataset.


# Solutions

1. The fundamental idea behind Support Vector Machines is to fit the widest possi‐
ble “street” between the classes. In other words, the goal is to have the largest pos‐
sible margin between the decision boundary that separates the two classes and
the training instances. When performing soft margin classification, the SVM
searches for a compromise between perfectly separating the two classes and hav‐
ing the widest possible street (i.e., a few instances may end up on the street).
Another key idea is to use kernels when training on nonlinear datasets.
2. After training an SVM, a support vector is any instance located on the “street” (see
the previous answer), including its border. The decision boundary is entirely
determined by the support vectors. Any instance that is not a support vector (i.e.,
is off the street) has no influence whatsoever; you could remove them, add more
instances, or move them around, and as long as they stay off the street they won’t
affect the decision boundary. Computing the predictions only involves the sup‐
port vectors, not the whole training set.

3. SVMs try to fit the largest possible “street” between the classes (see the first
answer), so if the training set is not scaled, the SVM will tend to neglect small
features (see Figure 5-2).
4. An SVM classifier can output the distance between the test instance and the deci‐
sion boundary, and you can use this as a confidence score. However, this score
cannot be directly converted into an estimation of the class probability. If you set
probability=True when creating an SVM in Scikit-Learn, then after training it
will calibrate the probabilities using Logistic Regression on the SVM’s scores
(trained by an additional five-fold cross-validation on the training data). This
will add the predict_proba() and predict_log_proba() methods to the SVM.
5. This question applies only to linear SVMs since kernelized SVMs can only use
the dual form. The computational complexity of the primal form of the SVM
problem is proportional to the number of training instances m, while the compu‐
tational complexity of the dual form is proportional to a number between m2
 and
m3
. So if there are millions of instances, you should definitely use the primal
form, because the dual form will be much too slow.
6. If an SVM classifier trained with an RBF kernel underfits the training set, there
might be too much regularization. To decrease it, you need to increase gamma or C
(or both).
7. Let’s call the QP parameters for the hard margin problem H′, f′, A′, and b′ (see
“Quadratic Programming” on page 167). The QP parameters for the soft margin
problem have m additional parameters (np = n + 1 + m) and m additional con‐
straints (nc
 = 2m). They can be defined like so:
 
 • H is equal to H′, plus m columns of 0s on the right and m rows of 0s at the
bottom: H =
H′ 0 ⋯
0 0
ۇٴ ⋮
• f is equal to f′ with m additional elements, all equal to the value of the hyper‐
parameter C.
• b is equal to b′ with m additional elements, all equal to 0.
• A is equal to A′, with an extra m × m identity matrix Im appended to the right,
–*I*m just below it, and the rest filled with 0s: A =
A′ Im
0 −Im

In [ ]:
For the solutions to exercises 8, 9, and 10, please see the Jupyter notebooks available
at https://github.com/ageron/handson-ml2.